In [ ]:
import os
import io
os.environ["CUDA_VISIBLE_DEVICES"]="1" 

In [ ]:
from keras.models import Model, Sequential,load_model
from keras import backend as K
from keras.utils import Sequence
from keras.layers.merge import *
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras import optimizers
from keras.optimizers import SGD
from keras.layers.merge import concatenate
import keras
import tensorflow as tf

from sklearn.model_selection import KFold

import numpy as np
import scipy as sp
import pandas as pd

from scipy.optimize import curve_fit, minimize

from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage import exposure
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from IPython.display import HTML

import time

from tensorflow.python import debug as tf_debug

import models.modded_unet as modded_unet
import models.loss

import data_loader
from tqdm import tqdm

In [ ]:
###################################
# TensorFlow wizardry
config = tf.ConfigProto()
 
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True
 
# Create a session with the above options specified.
keras.backend.tensorflow_backend.set_session(tf.Session(config=config))

In [ ]:
model_type = modded_unet

exp_name = "HoloUNet_cells"

max_x = 512
max_y = 512

## Define Model

In [ ]:
def build_unet(bb=None):
    unet = model_type.get_model(1,max_x,max_y,32)
    unet.compile(optimizer='adadelta', loss=models.loss.npcc_fft)

    return unet

In [ ]:
build_unet().summary()

## Load Dataset

In [ ]:
#Load Data
dl = data_loader.data_loader(max_x,max_y)
train_data_x, train_data_y, test_data_x, test_data_y, train_input_filelist, test_input_filelist = dl.get_bj7_ds()

## Train

In [ ]:
n_folds = 5
batch_size = 4

keras.backend.set_session(tf.Session())

In [ ]:
kf = KFold(n_splits=n_folds, shuffle=True)

i=0

for train_ind, val_ind in kf.split(train_data_x,train_data_y):
    train_x = train_data_x[train_ind]
    train_y = train_data_y[train_ind]
    
    val_x = train_data_x[val_ind]
    val_y = train_data_y[val_ind]

    checkpoint = ModelCheckpoint("weighted_models/{}_{}.h5".format(exp_name,i), monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    
    model = build_unet()
    model.fit(train_x,train_y,
          batch_size=batch_size,
          epochs=30,
          validation_data=(val_x,val_y),
          verbose=1,
          shuffle=True,
          callbacks=[checkpoint]) 
    
    i+=1

## Testing

In [ ]:
best_model = load_model('weighted_models/unet_fringe_contrast_fibroblast_fullfft_log_22-12-19--17-31_split1.h5.h5')

In [ ]:
for i in tqdm(np.arange(test_data_x.shape[0])):
    result = best_model.predict(test_data_x[i].reshape((1,max_x,max_y,2)))